In [1]:
import gym
from gym import error
from gym.utils import closer
from PIL import Image
import torch
from pypylon import pylon
import numpy as np
import cv2
from ctypes import *
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
import ray.rllib.agents.ppo as ppo
from ray.tune.logger import pretty_print
import glob
import matplotlib.pylab as pl
import matplotlib.pyplot as plt
import gym
from gym import error
from gym.utils import closer
from gym import spaces
from pypylon import pylon
import time
from matplotlib import cm
import random

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable
from torchvision import models, transforms
import time
import warnings

In [3]:
import imquality.brisque as brisque

In [4]:
%matplotlib inline

env_closer = closer.Closer()

warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
class CorningEnv(gym.Env):
    """The main OpenAI Gym class. It encapsulates an environment with
    arbitrary behind-the-scenes dynamics. An environment can be
    partially or fully observed.
    The main API methods that users of this class need to know are:
        step
        reset
        render
        close
        seed
    And set the following attributes:
        action_space: The Space object corresponding to valid actions
        observation_space: The Space object corresponding to valid observations
        reward_range: A tuple corresponding to the min and max possible rewards
    Note: a default reward range set to [-inf,+inf] already exists. Set it if you want a narrower range.
    The methods are accessed publicly as "step", "reset", etc...
    
    """
    metadata = {'render.modes': ['rgb_array']}
    
    def __init__(self, file_path = r'D:\RL_autofocus'):
        
        
        # Create an instant camera object with the camera device found first.
        self.camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        self.camera.Open()
        self.camera.AcquisitionFrameRateAbs.Value = 10
        self.camera.GainRaw.Value = 36
        self.camera.ExposureTimeRaw.Value = 30000
        self.camera.AcquisitionMode.SetValue('Continuous')
        self.camera.Width.Value = 417
        self.camera.Height.Value = 404
        print(self.camera.Width.GetValue(), self.camera.Height.GetValue())
        # code from https://github.com/basler/pypylon/blob/master/samples/opencv.py
        img = pylon.PylonImage()
        self.converter = pylon.ImageFormatConverter()
        # converting to opencv bgr format
        self.converter.OutputPixelFormat = pylon.PixelType_BGR8packed
        self.converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
        
        
        # corning lib
        self.lib = cdll.LoadLibrary(r"C:\Users\CIG\Documents\MATLAB\ComCasp64.dll")
        #Check if Maxim driver dll is loaded
        eCOMCaspErr = getattr(self.lib,'Casp_OpenCOM')
        print('eCOMCaspErr:', eCOMCaspErr(), self.lib.Casp_OpenCOM())
        
        
        # yolo
        self.model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', 
                                         force_reload=True, 
                                         pretrained=True)
        
        self.resnet152 = models.resnet152(pretrained=True)
        modules=list(self.resnet152.children())[:-1]
        self.resnet152=nn.Sequential(*modules)
        for p in self.resnet152.parameters():
            p.requires_grad = False
        
        self.action_space = spaces.MultiDiscrete([69, 99])
        self.observation_space = spaces.Box(0, 2, (2048, ))

        self.file_path = file_path
        self.step_rem_at_level = 10
        
    def grab_image(self):
        # https://github.com/basler/pypylon/blob/master/samples/opencv.py
        self.camera.StartGrabbing()
        while 1:
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
            if grabResult.GrabSucceeded():
                # Access the image data
                image = self.converter.Convert(grabResult)
                image = image.GetArray()
                #print(img[0])
                #print('shape:', img.shape)
                #plt.imshow(img)
                break
        self.camera.StopGrabbing()
        return image
        
    def lens_movement(self, action):
        x = c_double(action)
        self.lib.Casp_SetFocusVoltage(x)
        time.sleep(3)
    
    def find_class(self, results, obj = 'car'):
        len_of_class = len(results.pred)
        pred_class = [int(results.pred[0][i].numpy()[5]) for i in range(len(results.pred[0]))]
        #list_of_yolo_classes = list(zip(results.names, range(0, len(results.names))))
        pred_names = [results.names[i] for i in pred_class]
        if obj in pred_names:
            index = pred_names.index(obj)
            return results.pred[0][index].numpy()
        else:
            return np.array([])
        
    def take_features(self, img):
        preprocess = transforms.Compose([
                            transforms.Resize(256),
                            transforms.CenterCrop(224),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
        
        image = Image.fromarray(np.uint8(img)).convert('RGB')
        input_tensor = preprocess(image)
        input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
        
        # Run through the convolutional layers and resize the output.
        features_output = self.resnet152(input_batch)
        classifier_input = features_output.view(1, -1)
        
        return classifier_input.cpu().detach().numpy()[0]
    
    def reset(self):
        """Resets the environment to an initial state and returns an initial
        observation.
        Note that this function should not reset the environment's random
        number generator(s); random variables in the environment's state should
        be sampled independently between multiple calls to `reset()`. In other
        words, each call of `reset()` should yield an environment suitable for
        a new episode, independent of previous episodes.
        Returns:
            observation (object): the initial observation.
        """
        self.lens_movement(24.0)
        observation = self.grab_image()
        observation_feat = self.take_features(observation)
        self.step_rem_at_level = 10
        
        #print(observation_feat)
        #print(type(observation_feat), min(observation_feat), max(observation_feat), len(observation_feat))
        
        return observation_feat
           
    def step(self, action):
        """Run one timestep of the environment's dynamics. When end of
        episode is reached, you are responsible for calling `reset()`
        to reset this environment's state.
        Accepts an action and returns a tuple (observation, reward, done, info).
        Args:
            action (object): an action provided by the agent
        Returns:
            observation (object): agent's observation of the current environment
            reward (float) : amount of reward returned after previous action
            done (bool): whether the episode has ended, in which case further step() calls will return undefined results
            info (dict): contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)
        """
        self.step_rem_at_level-=1
        
        if self.step_rem_at_level == 0:
            done = True
        else:
            done = False
        
        action_lens_coarse = 0
        action_lens_fine = action[1]
        if action[0] < 24:
            action_lens_coarse = 24
        else:
            action_lens_coarse = action[0]
        action_lens = float("%s.%s"% (str(action_lens_coarse), str(action_lens_fine)))
        print(action_lens)
        
        self.lens_movement(action_lens)
        observation = self.grab_image()
        observation_feat = self.take_features(observation)
        
        prediction_yolo = self.model_yolo(observation)
        final_result_yolo = self.find_class(prediction_yolo)

        brisque_score = -10
        reward = brisque_score
        
        if final_result_yolo.size != 0:
            image = Image.fromarray(observation)
            image = image.crop((final_result_yolo[0], 
                             final_result_yolo[1], 
                             final_result_yolo[2], 
                             final_result_yolo[3]))
            brisque_score = round(brisque.score(image), 2)
            reward = - 0.01*brisque_score
            done = True

        im = Image.fromarray(observation)
        file_path = r'D:'
        filename = file_path + "\img_%s_%s.png" % (str(action_lens), str(brisque_score))
        im.save(filename)

        print('reward:', reward)
        return observation_feat, reward, done, {}

In [6]:
import ray
from ray.tune.registry import register_env

def env_creator(env_config):
    return  CorningEnv(env_config)

register_env("CorningEnv", env_creator)

In [7]:
ray.init()
config={
        "env": CorningEnv,
        "num_workers": 1,
        "entropy_coeff": 0.01,
        "rollout_fragment_length": 128,
        "train_batch_size": 128,
        "sgd_minibatch_size": 128,
        "lr": 0.00025,#tune.grid_search([0.01, 0.001, 0.0001]),
        # Smooth metrics over this many episodes.
        "metrics_smoothing_episodes": 10000,
    }

trainer = ppo.PPOTrainer(env= "CorningEnv", config=config)


2021-04-27 14:07:17,980	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
2021-04-27 14:07:20,038	INFO trainer.py:616 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-04-27 14:07:20,040	INFO trainer.py:643 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=16856) WARNING:tensorflow:From c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\tensorflow\python\compat\v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=16856) Instructions for updating:
(pid=16856) non-resource variables are not supported in the long term


(pid=16856) 417 404
(pid=16856) eCOMCaspErr: 0 0


(pid=16856) Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\CIG/.cache\torch\hub\master.zip
(pid=16856) 
(pid=16856)                  from  n    params  module                                  arguments                     
(pid=16856)   0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
(pid=16856)   1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
(pid=16856)   2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
(pid=16856)   3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
(pid=16856)   4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
(pid=16856)   5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
(pid=16856)   6           

In [8]:
trainer.restore(r'C:\Users\CIG\ray_results\OnlyLens_nofer2\checkpoint_34\checkpoint-34')

2021-04-27 14:07:33,973	INFO trainable.py:372 -- Restored on 10.16.102.19 from checkpoint: C:\Users\CIG\ray_results\OnlyLens_nofer2\checkpoint_34\checkpoint-34
2021-04-27 14:07:33,973	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 34, '_timesteps_total': None, '_time_total': 27849.84657382965, '_episodes_total': 2552}


In [ ]:
for i in range(100):
    # Perform one iteration of training the policy with PPO\n",
    result = trainer.train()
    print(pretty_print(result))
    print('-------', i, '-------')
# #     print(evaluation_results)
    if i % 80 == 0:
        checkpoint = trainer.save()
        print("checkpoint saved at", checkpoint)

(pid=16856) 2021-04-27 14:07:37,753	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!


(pid=16856) 48.25


(pid=16856) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=16856)   self.image = skimage.color.rgb2gray(self.image)


(pid=16856) reward: -0.149
(pid=16856) 47.8


(pid=16856) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=16856)   self.image = skimage.color.rgb2gray(self.image)


...

(pid=16856) reward: -0.4191
(pid=16856) 47.6


(pid=16856) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=16856)   self.image = skimage.color.rgb2gray(self.image)


(pid=16856) reward: -0.0808
(pid=16856) 47.96


(pid=16856) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=16856)   self.image = skimage.color.rgb2gray(self.image)


(pid=16856) reward: -0.1085
(pid=16856) 47.18


In [16]:
checkpoint = trainer.save()

(pid=13380) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=13380)   self.image = skimage.color.rgb2gray(self.image)


(pid=13380) reward: -0.6322
(pid=13380) 47.7


(pid=13380) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=13380)   self.image = skimage.color.rgb2gray(self.image)


(pid=13380) reward: -0.38180000000000003
(pid=13380) 47.25


(pid=13380) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=13380)   self.image = skimage.color.rgb2gray(self.image)


...

(pid=13380) reward: -0.5566
(pid=13380) 47.71


(pid=13380) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=13380)   self.image = skimage.color.rgb2gray(self.image)


(pid=13380) reward: -0.42110000000000003
(pid=13380) 48.69


(pid=13380) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=13380)   self.image = skimage.color.rgb2gray(self.image)


(pid=13380) reward: -0.6111


In [ ]:
#tensorboard --logdir=~/ray_results